## SSC Case Study: Initial data staging 

In [585]:
# Built-innp
from datetime import date
import time
from collections import Counter

# Other
import geopy
import pandas as pd
import numpy as np
import requests
pd.set_option('display.max_columns', None)

# Custom module
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)
from make_data import *

# Reload imported code 
%reload_ext autoreload
%autoreload 2

# Print all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [586]:
# load csv files

incidents_ssc = pd.read_csv("ssc2021_case_study_cer.csv")
incidents_extend = pd.read_csv("2021-03-31-comprehensive-incident-data.csv")
dates = pd.read_csv("Dates_2008_2020.csv")

In [587]:
dates

,Date_Key,Date_Full_Format,Day,Month,Year,Day_of_Week_Name,Day_of_Week_Num,Week_in_Year,Weekend
0,1.0,2008-01-01,1.0,1.0,2008.0,Tuesday,3.0,1.0,0.0
1,2.0,2008-01-02,2.0,1.0,2008.0,Wednesday,4.0,1.0,0.0
2,3.0,2008-01-03,3.0,1.0,2008.0,Thursday,5.0,1.0,0.0
3,4.0,2008-01-04,4.0,1.0,2008.0,Friday,6.0,1.0,0.0
4,5.0,2008-01-05,5.0,1.0,2008.0,Saturday,7.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
4792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4793,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [588]:
incidents_ssc

,Event Number,Occurred/Discovered Date and Time,Product Category,Product Type,Volume Released,Latitude,Longitude
0,INC2008-001,2008-01-02 11:30,Liquid,Crude Oil - Sweet,8.00,49.731350,-101.235570
1,INC2008-004,2008-01-23 6:05,Liquid,Crude Oil - Sweet,100.00,49.731350,-101.235570
2,INC2008-008,2008-01-26 10:45,Gas,Natural Gas - Sweet,NaN,58.012000,-122.694333
3,INC2008-009,2008-01-27 12:01,Miscellaneous,Potassium Hydroxide (caustic solution),NaN,56.143770,-120.516050
4,INC2008-128,2008-02-04 12:00,Gas,Natural Gas - Sweet,NaN,49.764920,-84.918240
...,...,...,...,...,...,...,...
718,INC2020-043,2020-04-26 8:34,Gas,Natural Gas - Sweet,1.00,56.705400,-111.403000
719,INC2020-045,2020-04-30 7:55,Liquid,Crude Oil - Sweet,50.00,51.649170,-108.211670
720,INC2020-050,2020-05-23 4:27,Gas,Natural Gas - Sweet,86.00,50.382480,-114.226414
721,INC2020-057,2020-06-12 11:23,Liquid,Crude Oil - Sweet,150.00,49.065679,-122.159714


In [589]:
incidents_extend

,Incident Number,Incident Types,Reported Date,Nearest Populated Centre,Province,Company,Status,Latitude,Approximate Volume Released (m3),Substance,Release Type,Significant,Year,Occurrence Date and Time,Discovered Date and Time,Detailed what happened,What happened category,Detailed why it happened,Why it happened category,Duration of interruption of pipeline operations,Pipeline or Facility Type,Activity being performed at time of incident,How the incident was discovered,Closed Date,Pipeline or facility equipment involved,Rupture,Incident type,Released substance type,Released volume (m3),Pipe body release,Residual effects on the environment,Source of Explosion,Source of Fire,Activity at time of fatality,Number of fatalities,Type of Injury,Number of individuals injured,Workdays lost,Work restricted by injury,Conditions that resulted in the operation beyond limits,Conditions that resulted in adverse effects on the environment,Number of people evacuated,Regulation,Pipeline Name,Pipeline outside diameter (NPS),Pipeline length (km),Substance carried,Facility Name,Facility Type,Facility latitude,Facility longitude,Country,Affects Company Property,Off Company Property,Affects Pipeline right-of-way,Affects off Pipeline right-of-way,Land Use,Population Density,Kilometre post,Emergency Level,Investigation Type,Was NEB Staff Deployed,Related NEB event number,Equipment or component involved,Design standard,Nominal pipe size,Material,Material grade,Schedule,Design wall thickness (mm),Custom design wall thickness (mm),Actual wall thickness (mm),Licensed maximum operating pressure (kPa),Restricted operating pressure (kPa),Actual operating pressure at time of failure (kPa),Designed depth of cover (m),Actual depth of cover (m),Year of manufacture,Year of installation,Year when put into service,Most recent cathodic protection reading at incident site (mV vs. Cu/CuSO4),Weld type,Seam type,Seam joining method,Seam clock position,Coating location,Coating type,Coating condition,Application method,Year when the coating was applied,Insulation installed,Repair type,Repair date,Equipment or component has never been inspected,Most recent inspection date for the failed equipment or component,Type of most recent inspection,Most recent inspection part of the routine inspection program,No maintenance done on this equipment or component,Date of the most recent maintenance work for the failed equipment or component,Most recent maintenance Type,Most recent maintenance work part of the routine maintenance program
0,INC2008-001,Release of Substance,1/2/2008,Cromer,Manitoba,Enbridge Pipelines Inc.,Closed,49.731350,8,Crude Oil - Sweet,Liquid,No,2008,2008/01/02 11:30:00 AM Central,2008/01/02 11:30:00 AM Central,"Damage or deterioration mechanism, Material Lo...",Corrosion and Cracking,"Job or system factors, Inadequate maintenance,...",Maintenance,NaN,NaN,Operation,Employee / Contractor (Onsite),1/11/2011,Yes,No,Release of Substance,Crude Oil - Sweet,8.00,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,OPR,EPI - Cromer Terminal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada,Yes,No,No,No,Unknown Land Use (Historical Data Migration),Unknown Population Density (Historical Data Mi...,NaN,Level I,Standard Review - 168 days,No,NaN,"Station/Facility, Pumping, Flange, Gasket/O-ring",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Type B Sleeve (Pressure Containing),1/11/2008,No,NaN,NaN,No,No,1/11/2008,Repair,No
1,INC2008-004,Release of Substance,1/23/2008,Cromer,Manitoba,Enbridge Pipelines Inc.,Closed,49.731350,100,Crude Oil - Sweet,Liquid,No,2008,2008/01/23 06:05:00 AM Mountain,2008/01/23 06:05:00 AM Mountain,"Damage or deterioration mechanism, Equipment, ...",Equipment Failure,"Job or system factors, Inadequate maintenance,...",Maintenance,NaN,NaN,Operation,Employee / Contractor (Onsite),8/10/2011,Yes,No,Release of Substance,Crude Oil - Sweet,100.00,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,OPR,EPI - Cromer Terminal,NaN,

In [590]:
# checking missing values - incidents_extend

def count_missing(table, col):
    count = table[col].isna().sum()
    percent = round(count / len(table)*100, 1)
    print(count, " -- ", percent, "% -- ", col)
    
for col in incidents_extend: 
    count_missing(incidents_extend, col)


0  --  0.0 % --  Incident Number
0  --  0.0 % --  Incident Types
0  --  0.0 % --  Reported Date
1  --  0.1 % --  Nearest Populated Centre
0  --  0.0 % --  Province
0  --  0.0 % --  Company
0  --  0.0 % --  Status
0  --  0.0 % --  Latitude
0  --  0.0 % --  Approximate Volume Released (m3)
0  --  0.0 % --  Substance
0  --  0.0 % --  Release Type
0  --  0.0 % --  Significant
0  --  0.0 % --  Year
303  --  21.9 % --  Occurrence Date and Time
2  --  0.1 % --  Discovered Date and Time
13  --  0.9 % --  Detailed what happened
0  --  0.0 % --  What happened category
14  --  1.0 % --  Detailed why it happened
0  --  0.0 % --  Why it happened category
737  --  53.4 % --  Duration of interruption of pipeline operations
661  --  47.9 % --  Pipeline or Facility Type
0  --  0.0 % --  Activity being performed at time of incident
116  --  8.4 % --  How the incident was discovered
7  --  0.5 % --  Closed Date
0  --  0.0 % --  Pipeline or facility equipment involved
0  --  0.0 % --  Rupture
0  --  0.0 %

In [591]:
# checking missing values - incididents_ssc

incidents_ssc.isna().sum()

Event Number                           0
Occurred/Discovered Date and Time      0
Product Category                       0
Product Type                           0
Volume Released                      251
Latitude                               0
Longitude                              0
dtype: int64

In [592]:
# checking frequencies 

Counter(incidents_ssc["Product Category"])

Counter({'Liquid': 79, 'Gas': 590, 'Miscellaneous': 54})

In [593]:
Counter(incidents_ssc["Product Type"])

Counter({'Crude Oil - Sweet': 43,
         'Natural Gas - Sweet': 477,
         'Potassium Hydroxide (caustic solution)': 1,
         'Natural Gas - Sour': 50,
         'Crude Oil - Synthetic': 7,
         'Propane': 21,
         'Diesel Fuel': 4,
         'Butane': 3,
         'Mixed HVP Hydrocarbons': 11,
         'Condensate': 7,
         'Amine': 4,
         'Glycol': 2,
         'Fuel Gas': 6,
         'Lube Oil': 8,
         'Sulphur': 12,
         'Gasoline': 4,
         'Sulphur Dioxide': 6,
         'Contaminated Water': 8,
         'Crude Oil - Sour': 3,
         'Jet Fuel': 1,
         'Waste Oil': 2,
         'Natural Gas Liquids': 22,
         'Pulp slurry': 1,
         'Hydrogen Sulphide': 7,
         'Water': 7,
         'Produced Water': 1,
         'Odourant': 1,
         'Potassium Carbonate': 1,
         'Drilling Fluid': 3})

In [594]:
# select subset of incident_extend features to include

incidents_extend_update = incidents_extend[['Incident Number', 'Incident Types', 'Nearest Populated Centre', 'Province', 
                                            'Company', 'Significant', 'Detailed what happened', 'What happened category', 
                                            'Detailed why it happened', 'Why it happened category', 
                                            'Activity being performed at time of incident', 'How the incident was discovered', 
                                            'Pipeline or facility equipment involved', 'Rupture', 'Pipe body release', 
                                            'Residual effects on the environment', 'Pipeline Name', 
                                            'Pipeline outside diameter (NPS)', 'Pipeline length (km)', 'Population Density',
                                            'Kilometre post', 'Emergency Level', 'Equipment or component involved',
                                            'Nominal pipe size', 'Material', 'Licensed maximum operating pressure (kPa)', 
                                            'Year of installation', 'Year when put into service', 'Repair type', 
                                            'Equipment or component has never been inspected', 
                                            'Most recent inspection date for the failed equipment or component',
                                            'Type of most recent inspection', 'Most recent inspection part of the routine inspection program',
                                            'No maintenance done on this equipment or component', 
                                            'Date of the most recent maintenance work for the failed equipment or component',
                                            'Most recent maintenance Type',
                                            'Most recent maintenance work part of the routine maintenance program']].copy()

In [595]:
incidents_extend.head()

,Incident Number,Incident Types,Reported Date,Nearest Populated Centre,Province,Company,Status,Latitude,Approximate Volume Released (m3),Substance,Release Type,Significant,Year,Occurrence Date and Time,Discovered Date and Time,Detailed what happened,What happened category,Detailed why it happened,Why it happened category,Duration of interruption of pipeline operations,Pipeline or Facility Type,Activity being performed at time of incident,How the incident was discovered,Closed Date,Pipeline or facility equipment involved,Rupture,Incident type,Released substance type,Released volume (m3),Pipe body release,Residual effects on the environment,Source of Explosion,Source of Fire,Activity at time of fatality,Number of fatalities,Type of Injury,Number of individuals injured,Workdays lost,Work restricted by injury,Conditions that resulted in the operation beyond limits,Conditions that resulted in adverse effects on the environment,Number of people evacuated,Regulation,Pipeline Name,Pipeline outside diameter (NPS),Pipeline length (km),Substance carried,Facility Name,Facility Type,Facility latitude,Facility longitude,Country,Affects Company Property,Off Company Property,Affects Pipeline right-of-way,Affects off Pipeline right-of-way,Land Use,Population Density,Kilometre post,Emergency Level,Investigation Type,Was NEB Staff Deployed,Related NEB event number,Equipment or component involved,Design standard,Nominal pipe size,Material,Material grade,Schedule,Design wall thickness (mm),Custom design wall thickness (mm),Actual wall thickness (mm),Licensed maximum operating pressure (kPa),Restricted operating pressure (kPa),Actual operating pressure at time of failure (kPa),Designed depth of cover (m),Actual depth of cover (m),Year of manufacture,Year of installation,Year when put into service,Most recent cathodic protection reading at incident site (mV vs. Cu/CuSO4),Weld type,Seam type,Seam joining method,Seam clock position,Coating location,Coating type,Coating condition,Application method,Year when the coating was applied,Insulation installed,Repair type,Repair date,Equipment or component has never been inspected,Most recent inspection date for the failed equipment or component,Type of most recent inspection,Most recent inspection part of the routine inspection program,No maintenance done on this equipment or component,Date of the most recent maintenance work for the failed equipment or component,Most recent maintenance Type,Most recent maintenance work part of the routine maintenance program
0,INC2008-001,Release of Substance,1/2/2008,Cromer,Manitoba,Enbridge Pipelines Inc.,Closed,49.731350,8,Crude Oil - Sweet,Liquid,No,2008,2008/01/02 11:30:00 AM Central,2008/01/02 11:30:00 AM Central,"Damage or deterioration mechanism, Material Lo...",Corrosion and Cracking,"Job or system factors, Inadequate maintenance,...",Maintenance,NaN,NaN,Operation,Employee / Contractor (Onsite),1/11/2011,Yes,No,Release of Substance,Crude Oil - Sweet,8.0,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,OPR,EPI - Cromer Terminal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada,Yes,No,No,No,Unknown Land Use (Historical Data Migration),Unknown Population Density (Historical Data Mi...,NaN,Level I,Standard Review - 168 days,No,NaN,"Station/Facility, Pumping, Flange, Gasket/O-ring",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Type B Sleeve (Pressure Containing),1/11/2008,No,NaN,NaN,No,No,1/11/2008,Repair,No
1,INC2008-004,Release of Substance,1/23/2008,Cromer,Manitoba,Enbridge Pipelines Inc.,Closed,49.731350,100,Crude Oil - Sweet,Liquid,No,2008,2008/01/23 06:05:00 AM Mountain,2008/01/23 06:05:00 AM Mountain,"Damage or deterioration mechanism, Equipment, ...",Equipment Failure,"Job or system factors, Inadequate maintenance,...",Maintenance,NaN,NaN,Operation,Employee / Contractor (Onsite),8/10/2011,Yes,No,Release of Substance,Crude Oil - Sweet,100.0,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,OPR,EPI - Cromer Terminal,NaN,Na

In [596]:
incidents_ssc.head()

,Event Number,Occurred/Discovered Date and Time,Product Category,Product Type,Volume Released,Latitude,Longitude
0,INC2008-001,2008-01-02 11:30,Liquid,Crude Oil - Sweet,8.0,49.73135,-101.235570
1,INC2008-004,2008-01-23 6:05,Liquid,Crude Oil - Sweet,100.0,49.73135,-101.235570
2,INC2008-008,2008-01-26 10:45,Gas,Natural Gas - Sweet,NaN,58.01200,-122.694333
3,INC2008-009,2008-01-27 12:01,Miscellaneous,Potassium Hydroxide (caustic solution),NaN,56.14377,-120.516050
4,INC2008-128,2008-02-04 12:00,Gas,Natural Gas - Sweet,NaN,49.76492,-84.918240


In [597]:
incidents_extend.isna().mean().to_frame(name = 'percent_missing').query('percent_missing < 0.4')

,percent_missing
Incident Number,0.000000
Incident Types,0.000000
Reported Date,0.000000
Nearest Populated Centre,0.000724
Province,0.000000
Company,0.000000
Status,0.000000
Latitude,0.000000
Approximate Volume Released (m3),0.000000
Substance,0.000000


In [598]:
incidents_extend_update 

,Incident Number,Incident Types,Nearest Populated Centre,Province,Company,Significant,Detailed what happened,What happened category,Detailed why it happened,Why it happened category,Activity being performed at time of incident,How the incident was discovered,Pipeline or facility equipment involved,Rupture,Pipe body release,Residual effects on the environment,Pipeline Name,Pipeline outside diameter (NPS),Pipeline length (km),Population Density,Kilometre post,Emergency Level,Equipment or component involved,Nominal pipe size,Material,Licensed maximum operating pressure (kPa),Year of installation,Year when put into service,Repair type,Equipment or component has never been inspected,Most recent inspection date for the failed equipment or component,Type of most recent inspection,Most recent inspection part of the routine inspection program,No maintenance done on this equipment or component,Date of the most recent maintenance work for the failed equipment or component,Most recent maintenance Type,Most recent maintenance work part of the routine maintenance program
0,INC2008-001,Release of Substance,Cromer,Manitoba,Enbridge Pipelines Inc.,No,"Damage or deterioration mechanism, Material Lo...",Corrosion and Cracking,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,EPI - Cromer Terminal,NaN,NaN,Unknown Population Density (Historical Data Mi...,NaN,Level I,"Station/Facility, Pumping, Flange, Gasket/O-ring",NaN,NaN,NaN,NaN,NaN,Type B Sleeve (Pressure Containing),No,NaN,NaN,No,No,1/11/2008,Repair,No
1,INC2008-004,Release of Substance,Cromer,Manitoba,Enbridge Pipelines Inc.,No,"Damage or deterioration mechanism, Equipment, ...",Equipment Failure,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,EPI - Cromer Terminal,NaN,NaN,Unknown Population Density (Historical Data Mi...,Mile Post 595.8,Level I,"Station/Facility, Pumping, Flange, Gasket/O-ring",NPS 30,NaN,0.0,NaN,NaN,Replacement,No,NaN,NaN,No,No,NaN,Replacement,No
2,INC2008-008,Release of Substance,Fort Nelson,British Columbia,"Westcoast Energy Inc., carrying on business as...",No,"Damage or deterioration mechanism, Constructio...",Defect and Deterioration,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,NaN,NaN,NaN,Unknown Population Density (Historical Data Mi...,NaN,Level I,"Station/Facility, Compression, Flange",NaN,NaN,0.0,NaN,NaN,NaN,No,NaN,NaN,No,No,NaN,NaN,No
3,INC2008-009,"Release of Substance, Adverse Environmental Ef...",Taylor,British Columbia,"Westcoast Energy Inc., carrying on business as...",Yes,"Damage or deterioration mechanism, Other Cause...",Equipment Failure,"Job or system factors, Inadequate tools and eq...","Maintenance, Standards and Procedures, Tools a...",Operation,NaN,Yes,No,No,Yes,NaN,NaN,NaN,Unknown Population Density (Historical Data Mi...,NaN,Level I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,No,NaN,NaN,No
4,INC2008-016,Release of Substance,Beiseker,Alberta,Foothills Pipe Lines Ltd.,No,"Damage or deterioration mechanism, Material or...",Defect and Deterioration,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,NaN,NaN,NaN,Unknown Population Density (Historical Data Mi...,Mainline Valve + Offset (km) MLV 61-121-NC,Level I,"Pipeline, Valve, Ball, Bonnet",NPS 30,NaN,0.0,NaN,NaN,Rebuilt,No,NaN,NaN,No,No,3/25/2008,Repair,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,INC2020-059,Fire,Rolphton,Ontario,TransCanada PipeLines Limited,No,"Damage or deterioration mechanism, Structural ...","Defect and Deterioration, External Interference","Job or system factors, Excessive wear and tear...",Maintenance,Maintenance,Employee / Contractor (Onsite),No,No,NaN,No,Line 1200-1,914.4,431.707658,10 or

In [599]:
# merge incidents table with extended incident features 

incidents = pd.merge(incidents_ssc, incidents_extend_update,  how='left', left_on=['Event Number'], 
                     right_on = ['Incident Number'])

In [600]:
# drop extra key

del incidents["Incident Number"]

In [601]:
# create separate date column 

incidents['Occurred Date'] = incidents['Occurred/Discovered Date and Time'].str[:10]

In [602]:
# merge incidents table with date table  

incidents = pd.merge(incidents, dates,  how='left', left_on=['Occurred Date'], right_on = ['Date_Full_Format'])

In [603]:
# reordering columns

incidents = incidents[['Event Number', 'Occurred/Discovered Date and Time', 'Occurred Date', 'Day', 'Month', 'Year', 
                       'Day_of_Week_Name', 'Day_of_Week_Num', 'Week_in_Year', 'Weekend', 'Latitude', 'Longitude', 
                       'Nearest Populated Centre', 'Province', 'Population Density', 'Incident Types', 'Company', 'Significant',
                       'Product Category', 'Product Type', 'Volume Released', 'Detailed what happened', 'What happened category', 
                       'Detailed why it happened', 'Why it happened category', 'Activity being performed at time of incident',
                       'How the incident was discovered', 'Pipeline or facility equipment involved', 'Rupture', 'Pipe body release',
                       'Residual effects on the environment', 'Pipeline Name', 'Pipeline outside diameter (NPS)', 
                       'Pipeline length (km)', 'Kilometre post', 'Emergency Level', 'Equipment or component involved',
                       'Nominal pipe size', 'Material', 'Licensed maximum operating pressure (kPa)', 'Year of installation', 
                       'Year when put into service', 'Repair type', 'Equipment or component has never been inspected', 
                       'Most recent inspection date for the failed equipment or component', 'Type of most recent inspection', 
                       'Most recent inspection part of the routine inspection program',
                       'No maintenance done on this equipment or component', 
                       'Date of the most recent maintenance work for the failed equipment or component', 
                       'Most recent maintenance Type', 'Most recent maintenance work part of the routine maintenance program']].copy()


In [604]:
# final incidents table

incidents

,Event Number,Occurred/Discovered Date and Time,Occurred Date,Day,Month,Year,Day_of_Week_Name,Day_of_Week_Num,Week_in_Year,Weekend,Latitude,Longitude,Nearest Populated Centre,Province,Population Density,Incident Types,Company,Significant,Product Category,Product Type,Volume Released,Detailed what happened,What happened category,Detailed why it happened,Why it happened category,Activity being performed at time of incident,How the incident was discovered,Pipeline or facility equipment involved,Rupture,Pipe body release,Residual effects on the environment,Pipeline Name,Pipeline outside diameter (NPS),Pipeline length (km),Kilometre post,Emergency Level,Equipment or component involved,Nominal pipe size,Material,Licensed maximum operating pressure (kPa),Year of installation,Year when put into service,Repair type,Equipment or component has never been inspected,Most recent inspection date for the failed equipment or component,Type of most recent inspection,Most recent inspection part of the routine inspection program,No maintenance done on this equipment or component,Date of the most recent maintenance work for the failed equipment or component,Most recent maintenance Type,Most recent maintenance work part of the routine maintenance program
0,INC2008-001,2008-01-02 11:30,2008-01-02,2.0,1.0,2008.0,Wednesday,4.0,1.0,0.0,49.731350,-101.235570,Cromer,Manitoba,Unknown Population Density (Historical Data Mi...,Release of Substance,Enbridge Pipelines Inc.,No,Liquid,Crude Oil - Sweet,8.00,"Damage or deterioration mechanism, Material Lo...",Corrosion and Cracking,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,EPI - Cromer Terminal,NaN,NaN,NaN,Level I,"Station/Facility, Pumping, Flange, Gasket/O-ring",NaN,NaN,NaN,NaN,NaN,Type B Sleeve (Pressure Containing),No,NaN,NaN,No,No,1/11/2008,Repair,No
1,INC2008-004,2008-01-23 6:05,2008-01-23,23.0,1.0,2008.0,Wednesday,4.0,4.0,0.0,49.731350,-101.235570,Cromer,Manitoba,Unknown Population Density (Historical Data Mi...,Release of Substance,Enbridge Pipelines Inc.,No,Liquid,Crude Oil - Sweet,100.00,"Damage or deterioration mechanism, Equipment, ...",Equipment Failure,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,EPI - Cromer Terminal,NaN,NaN,Mile Post 595.8,Level I,"Station/Facility, Pumping, Flange, Gasket/O-ring",NPS 30,NaN,0.0,NaN,NaN,Replacement,No,NaN,NaN,No,No,NaN,Replacement,No
2,INC2008-008,2008-01-26 10:45,2008-01-26,26.0,1.0,2008.0,Saturday,7.0,4.0,1.0,58.012000,-122.694333,Fort Nelson,British Columbia,Unknown Population Density (Historical Data Mi...,Release of Substance,"Westcoast Energy Inc., carrying on business as...",No,Gas,Natural Gas - Sweet,NaN,"Damage or deterioration mechanism, Constructio...",Defect and Deterioration,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,NaN,NaN,NaN,NaN,Level I,"Station/Facility, Compression, Flange",NaN,NaN,0.0,NaN,NaN,NaN,No,NaN,NaN,No,No,NaN,NaN,No
3,INC2008-009,2008-01-27 12:01,2008-01-27,27.0,1.0,2008.0,Sunday,1.0,5.0,1.0,56.143770,-120.516050,Taylor,British Columbia,Unknown Population Density (Historical Data Mi...,"Release of Substance, Adverse Environmental Ef...","Westcoast Energy Inc., carrying on business as...",Yes,Miscellaneous,Potassium Hydroxide (caustic solution),NaN,"Damage or deterioration mechanism, Other Cause...",Equipment Failure,"Job or system factors, Inadequate tools and eq...","Maintenance, Standards and Procedures, Tools a...",Operation,NaN,Yes,No,No,Yes,NaN,NaN,NaN,NaN,Level I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,No,No,NaN,NaN,No
4,INC2008-128,2008-02-04 12:00,2008-02-04,4.0,2.0,2008.0,Monday,2.0,6.0,0.0,49.764920,-84.918240,Hearst,Ontario,Unknown Population Density (Historical Data Mi...,Release of Substance,TransCanada PipeLines Limited,No,Gas,Natural Gas - Sweet,NaN,"Damage or deterioration mechanism, Cracking, F...",C

In [605]:
# missing values 

for col in incidents: 
    count_missing(incidents, col)

0  --  0.0 % --  Event Number
0  --  0.0 % --  Occurred/Discovered Date and Time
0  --  0.0 % --  Occurred Date
0  --  0.0 % --  Day
0  --  0.0 % --  Month
0  --  0.0 % --  Year
0  --  0.0 % --  Day_of_Week_Name
0  --  0.0 % --  Day_of_Week_Num
0  --  0.0 % --  Week_in_Year
0  --  0.0 % --  Weekend
0  --  0.0 % --  Latitude
0  --  0.0 % --  Longitude
0  --  0.0 % --  Nearest Populated Centre
0  --  0.0 % --  Province
0  --  0.0 % --  Population Density
0  --  0.0 % --  Incident Types
0  --  0.0 % --  Company
0  --  0.0 % --  Significant
0  --  0.0 % --  Product Category
0  --  0.0 % --  Product Type
251  --  34.7 % --  Volume Released
7  --  1.0 % --  Detailed what happened
0  --  0.0 % --  What happened category
7  --  1.0 % --  Detailed why it happened
0  --  0.0 % --  Why it happened category
0  --  0.0 % --  Activity being performed at time of incident
48  --  6.6 % --  How the incident was discovered
0  --  0.0 % --  Pipeline or facility equipment involved
0  --  0.0 % --  Rupture

In [606]:
# export updated incidents data 

incidents.to_csv("Incidents_Updated.csv",index=False)

## Meteorological Information

Web application - https://climate.weather.gc.ca/historical_data/search_historic_data_e.html

Canada Weather API - https://api.weather.gc.ca/openapi?f=html#/ and https://api.weather.gc.ca/

In [607]:
%%time 

weather_api_url = 'https://api.weather.gc.ca'
provinces = incidents['Province'].unique().tolist()

# Get all weather stations
weather_stations = scrape_weather_stations(provinces)
weather_stations.to_csv('weather_stations.csv')

incidents['Occurred Date'] = pd.to_datetime(incidents['Occurred Date'], format="%Y-%m-%d").dt.date

Wall time: 7.09 s


In [609]:
%%time

# Get weather information for each incident
incidents_weather = incidents.apply(lambda x: weather_data(x['Latitude'], x['Longitude'], x['Occurred Date'], weather_stations), axis = 1)
incidents_weather = pd.concat(incidents_weather.to_list()).reset_index(drop = True)
incidents = pd.concat([incidents, incidents_weather], axis = 1)

incidents.head(5)

Wall time: 5min 9s


,Event Number,Occurred/Discovered Date and Time,Occurred Date,Day,Month,Year,Day_of_Week_Name,Day_of_Week_Num,Week_in_Year,Weekend,Latitude,Longitude,Nearest Populated Centre,Province,Population Density,Incident Types,Company,Significant,Product Category,Product Type,Volume Released,Detailed what happened,What happened category,Detailed why it happened,Why it happened category,Activity being performed at time of incident,How the incident was discovered,Pipeline or facility equipment involved,Rupture,Pipe body release,Residual effects on the environment,Pipeline Name,Pipeline outside diameter (NPS),Pipeline length (km),Kilometre post,Emergency Level,Equipment or component involved,Nominal pipe size,Material,Licensed maximum operating pressure (kPa),Year of installation,Year when put into service,Repair type,Equipment or component has never been inspected,Most recent inspection date for the failed equipment or component,Type of most recent inspection,Most recent inspection part of the routine inspection program,No maintenance done on this equipment or component,Date of the most recent maintenance work for the failed equipment or component,Most recent maintenance Type,Most recent maintenance work part of the routine maintenance program,STATION_NAME,DISTANCE_WEATHER_STATION_KM,CLIMATE_IDENTIFIER,MEAN_TEMPERATURE,MEAN_TEMPERATURE_FLAG,MIN_TEMPERATURE,MIN_TEMPERATURE_FLAG,MAX_TEMPERATURE,MAX_TEMPERATURE_FLAG,TOTAL_PRECIPITATION,TOTAL_PRECIPITATION_FLAG,TOTAL_RAIN,TOTAL_RAIN_FLAG,TOTAL_SNOW,TOTAL_SNOW_FLAG,SNOW_ON_GROUND,SNOW_ON_GROUND_FLAG,DIRECTION_MAX_GUST,DIRECTION_MAX_GUST_FLAG,SPEED_MAX_GUST,SPEED_MAX_GUST_FLAG,COOLING_DEGREE_DAYS,COOLING_DEGREE_DAYS_FLAG,HEATING_DEGREE_DAYS,HEATING_DEGREE_DAYS_FLAG,MIN_REL_HUMIDITY,MIN_REL_HUMIDITY_FLAG,MAX_REL_HUMIDITY,MAX_REL_HUMIDITY_FLAG
0,INC2008-001,2008-01-02 11:30,2008-01-02,2.0,1.0,2008.0,Wednesday,4.0,1.0,0.0,49.73135,-101.235570,Cromer,Manitoba,Unknown Population Density (Historical Data Mi...,Release of Substance,Enbridge Pipelines Inc.,No,Liquid,Crude Oil - Sweet,8.0,"Damage or deterioration mechanism, Material Lo...",Corrosion and Cracking,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,EPI - Cromer Terminal,NaN,NaN,NaN,Level I,"Station/Facility, Pumping, Flange, Gasket/O-ring",NaN,NaN,NaN,NaN,NaN,Type B Sleeve (Pressure Containing),No,NaN,NaN,No,No,1/11/2008,Repair,No,ELKHORN 2 EAST,21.429348,5010QFQ,-14.5,None,-27,None,-2,None,0,None,0,None,0,None,12,None,None,None,None,None,0,None,32.5,None,None,None,None,None
1,INC2008-004,2008-01-23 6:05,2008-01-23,23.0,1.0,2008.0,Wednesday,4.0,4.0,0.0,49.73135,-101.235570,Cromer,Manitoba,Unknown Population Density (Historical Data Mi...,Release of Substance,Enbridge Pipelines Inc.,No,Liquid,Crude Oil - Sweet,100.0,"Damage or deterioration mechanism, Equipment, ...",Equipment Failure,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,EPI - Cromer Terminal,NaN,NaN,Mile Post 595.8,Level I,"Station/Facility, Pumping, Flange, Gasket/O-ring",NPS 30,NaN,0.0,NaN,NaN,Replacement,No,NaN,NaN,No,No,NaN,Replacement,No,ELKHORN 2 EAST,21.429348,5010QFQ,-20.5,None,-25.5,None,-15.5,None,0,None,0,None,0,None,11,None,None,None,None,None,0,None,38.5,None,None,None,None,None
2,INC2008-008,2008-01-26 10:45,2008-01-26,26.0,1.0,2008.0,Saturday,7.0,4.0,1.0,58.01200,-122.694333,Fort Nelson,British Columbia,Unknown Population Density (Historical Data Mi...,Release of Substance,"Westcoast Energy Inc., carrying on business as...",No,Gas,Natural Gas - Sweet,NaN,"Damage or deterioration mechanism, Constructio...",Defect and Deterioration,"Job or system factors, Inadequate maintenance,...",Maintenance,Operation,Employee / Contractor (Onsite),Yes,No,No,No,NaN,NaN,NaN,NaN,Level I,"Station/Facility, Compression, Flange",NaN,NaN,0.0,NaN,NaN,NaN,No,NaN,NaN,No,No,NaN,NaN,No,FORT NELSON A,58.005254,1192940,-20.5,None,-28.7,None,-12.2,None,3.2,N

In [ ]:
# Check incidents that have missing weather information 
incidents[incidents['MIN_TEMPERATURE'].isnull()]

In [611]:
# Save
incidents.to_csv('Incidents_Updated.csv', index = False)